In [ ]:
import sys;
sys.path.insert(0, '..')

## Chapter 6 Code Snippets and Listings

### Finding good and bad outcomes with oracles (section 6.1.1)

Assume we want to find one special item in a list of $N = 8$ items, but we do not know where the item is. We can represent each of the items as an outcome of a quantum state with $n = 3$ qubits. Any randomly selected outcome has the same likelihood of being the good outcome. We represent this with equal amplitudes for each outcome:

In [ ]:
from math import sqrt

n = 3
state = [1/sqrt(2**n) for _ in range(2**n)]

In [ ]:
from util import print_state_table

print_state_table(state)

Let's look at an example of the classical implementation of a phase oracle from chapter 8. The function `predicate` identifies good outcome 3 as the only good outcome.

In [ ]:
predicate = lambda k: True if k == 3 else False

We can use this predicate for the classical implementation of a phase oracle:

In [ ]:
def oracle(state, predicate):
    for item in range(len(state)):
        if predicate(item):
            state[item] *= -1
            
oracle(state, predicate)

In [ ]:
print_state_table(state)

### Inversion by the mean (section 6.1.2)

Listing 6.1 Classical implementation of the inversion operator

In [ ]:
def inner(v1, v2):
    assert(len(v1) == len(v2))
    return sum(z1*z2.conjugate() for z1, z2 in zip(v1, v2))

def inversion(original, current):
    proj = inner(original, current)
    for k in range(len(current)):
        current[k] = 2*proj*original[k] - current[k]

Previous example three-qubit state obtained by applying an oracle that tags the outcome 3 to a state where the amplitudes are in equal superposition:

In [ ]:
n = 3
state = [1/sqrt(2**n) for _ in range(2**n)]

s = state.copy() # Copy the original state to use for the inversion

oracle(state, predicate)

Now, we can apply the inversion operator that reflects the current state into the original state (the state before the oracle was applied):

In [ ]:
inversion(s, state) # The parameter s is the initial state before the oracle was applied

In [ ]:
print_state_table(state)

Let's look at another example, with a random $n = 3$ qubit state and good outcome 5:

In [ ]:
from util import generate_state

n = 3
state = generate_state(n)

In [ ]:
print_state_table(state)

In [ ]:
s = state.copy()

predicate = lambda k: True if k == 5 else False
oracle(state, predicate)

In [ ]:
print_state_table(state)

Next, we perform the inversion:

In [ ]:
inversion(s, state)

In [ ]:
print_state_table(state)

**Visualizing inversion by the mean**

let's create a $n = 3$ qubit state in equal superposition and apply an oracle for good outcome 3:

In [ ]:
n = 3
state = [1/sqrt(2**n) for _ in range(2**n)]

s = state.copy()

predicate = lambda k: True if k == 3 else False
oracle(state, predicate)

We can check that the mean of the amplitudes is equal to the quantity defined above for bad outcomes $k$:

In [ ]:
from util import is_close

amplitude_mean = sum(state)/2**n

proj = inner(s, state)
for k in range(len(state)):
    if k != 3:
        assert is_close(proj*state[k], amplitude_mean)

Now, we can simulate the inversion by the mean with the following Python code:

In [ ]:
for k in range(len(state)):
    state[k] = 2*amplitude_mean-state[k]

In [ ]:
print_state_table(state)

### Putting it together: the Grover iterate (section 6.1.3)

Listing 6.2 Classical implementation of the Grover iterate

In [ ]:
from math import cos

def grover_sim(state, predicate, iterations):
    s = state.copy()

    # Use the probability of measuring a good outcome to define an angle theta
    p = sum([abs(s[k])**2 for k in items])
    theta = asin(sqrt(p))
    assert is_close(inner(s, state), 1)

    for it in range(1, iterations + 1):
        oracle(state, predicate)
        inversion(s, state)
        # The inner product after operator A is applied and the state after j Grover iterations is cos(2j theta)
        assert is_close(inner(s, state), cos(2*it*theta))

        p = sum([abs(state[k])**2 for k in items]) # Find the new probability of measuring a good outcome
        assert is_close(p, sin((2*it + 1)*theta)**2) # Check that the probability of good outcomes is sin^2((2j+1)theta)

In the case that the operator $A$ prepares a state with a uniform distribution, the magnitudes of good outcomes are given by the function below, where the parameter `n` is the number of qubits, `L` is the number of good outcomes, and `j` is the number of iterations.

In [ ]:
from math import sin, asin

def target_amplitude_uniform(n, l, j):
    theta = asin(sqrt(l/2**n))
    return sin((2*j+1)*theta)/sqrt(l)

Let's apply one iteration of the Grover iterate to our example state, where $n = 3$ and the outcome 3 is the good outcome:

In [ ]:
n = 3
items = [3]
predicate = lambda i: True if i in items else False

state = [1/sqrt(2**n) for _ in range(2**n)]

grover_sim(state, predicate, iterations = 1)

assert is_close(state[items[0]], target_amplitude_uniform(3, 1, 1))

In [ ]:
print_state_table(state)

Let's apply another iteration:

In [ ]:
n = 3
items = [3]
predicate = lambda i: True if i in items else False

state = [1/sqrt(2**n) for _ in range(2**n)]

grover_sim(state, predicate, iterations = 2)

assert is_close(state[items[0]], target_amplitude_uniform(3, 1, 2))

In [ ]:
print_state_table(state)

Let's try three iterations:

In [ ]:
n = 3
items = [3]
predicate = lambda i: True if i in items else False

state = [1/sqrt(2**n) for _ in range(2**n)]

grover_sim(state, predicate, iterations = 3)

assert is_close(state[items[0]], target_amplitude_uniform(3, 1, 3))

In [ ]:
print_state_table(state)

Using the number of good outcomes, `L = len(items)`, and the total number of outcomes, `2**n`, we can find the optimal number of iterations with:

In [ ]:
from math import floor, pi

num_iterations = int(floor(pi/4*sqrt(2**n/len(items))))

Let's look at another example, this time using a random two-qubit state and good outcome 1:

In [ ]:
n = 3
items = [0, 1]
predicate = lambda i: True if i in items else False
iterations = int(floor(pi/4*sqrt(2**n/len(items))))

state = generate_state(n)
grover_sim(state, predicate, iterations)

In [ ]:
print_state_table(state)

### A new implementation of the inversion operator (section 6.1.4)

In this section, we will use a randomly generated operator $A$. To create such an operator for $n$ qubits, we will use the function `random_transformation` defined in util.py. 

Let's define a random transformation (and its inverse) for $n = 3$ qubits:

In [ ]:
from util import random_transformation

n = 3
f = random_transformation(n)
A = f[0]
A_inverse = f[1]

Let's look at the state prepared by this random operator:

In [ ]:
from sim_core import init_state

state = init_state(n)
f[0](state)

In [ ]:
print_state_table(state)

Listing 6.3 Function to perform the inversion operation using the operator $A$

In [ ]:
from math import log2

def inversion_0_transformation(f, state):
    n = int(log2(len(state)))

    transform = f[0]
    inverse_transform = f[1]

    inverse_transform(state)
    inversion(init_state(n), state)
    transform(state)

Let's use the same operator $A$ we used to create the state above and apply an oracle for the good outcome 3:

In [ ]:
predicate = lambda k: True if k == 3 else False
oracle(state, predicate)

In [ ]:
print_state_table(state)

Now, we can apply the inversion operation to the state:

In [ ]:
inversion_0_transformation(f, state)

In [ ]:
print_state_table(state)

### Quantum Oracle (section 6.2.1)

Listing 6.4 Function to create a phase oracle quantum circuit for a given number of qubits `n` and a set of good outcomes `items`

In [ ]:
from sim_circuit import *

def is_bit_not_set(m, k):
    return not (m & (1 << k))

def phase_oracle_match(n, items):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for m in items:
        for i in range(n):
            if is_bit_not_set(m, i):
                qc.x(q[i])

        qc.mcp(pi, [q[i] for i in range(len(q) - 1)], q[len(q) - 1])

        for i in range(n):
            if is_bit_not_set(m, i):
                qc.x(q[i])
    return qc

### The inversion operator (section 6.2.2)

Listing 6.5 Function to create a circuit for `n` qubits that multiplies outcome 0 by -1

In [ ]:
def inversion_0_circuit(n):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for i in range(n):
        qc.x(q[i])

    qc.mcp(pi, [q[i] for i in range(n - 1)], q[n - 1])

    for i in range(n):
        qc.x(q[i])

    return qc

Listing 6.6 Function to create the inversion circuit

In [ ]:
def inversion_circuit(A):
    n = sum(A.regs)
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    qc.append(A.inverse(), q)

    qc.append(inversion_0_circuit(n), q)

    qc.append(A, q)

    return qc

### Grover iterate (section 6.2.3)

Listing 6.7 Function to create the Grover iterate circuit for a given oracle `O` and operator `A`

In [ ]:
def grover_iterate_circuit(A, O):
    n = sum(O.regs)
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    qc.append(O, q)

    qc.append(inversion_circuit(A), q)

    return qc

### Putting it all together: Grover's Algorithm (section 6.2.4)

Listing 6.8 Function to create the magnitude amplification circuit

In [ ]:
def grover_circuit(A, O, iterations):
    n = sum(A.regs)
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)
    
    qc.append(A, q)

    for i in range(1, iterations + 1):
        qc.append(grover_iterate_circuit(A, O), q)
        qc.report(f'iteration_{i}')

    return qc

We can use the function `uniform` from chapter 4 to create a circuit $A$ that prepares a state with equal magnitudes:

In [ ]:
def uniform(n):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for i in range(len(q)):
        qc.h(q[i])

    return qc

With `uniform` as our operator $A$, $n = 3$ qubits, and good outcomes 1, 3, and 7, we can define the following circuit:

In [ ]:
n = 3
items = [1, 3, 7]
num_iterations = int(floor(pi/4*sqrt(2**n/len(items))))

qc = grover_circuit(uniform(n), phase_oracle_match(n, items), num_iterations)

In [ ]:
from util_qiskit import print_circuit

print_circuit(qc)

We can check the amplitudes at each step using the reports generated with the following code:

In [ ]:
for i in range(1, num_iterations + 1):
    for m in items:
        assert is_close(qc.reports[f'iteration_{i}'][2][m], (-1)**i * target_amplitude_uniform(n, len(items), i))

In [ ]:
print_state_table(qc.run())